In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import cv2
import os
import sys
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten, BatchNormalization, Conv2DTranspose, Add, Concatenate
from keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img
from tensorflow.keras import regularizers
from tensorflow.image import ssim
from tensorflow.keras.applications import VGG16
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.backend import repeat_elements
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras import backend as K

In [18]:
def prepare_dataframe(input_folder):
    image_dir = input_folder
    file_path =  [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(('JPG','PNG','JPEG','jpeg','.jpg'))]
    df = pd.DataFrame({
        'filename' : file_path,
        'class' : [0] * len(file_path)
    })
    
    return df

In [19]:
df_train=prepare_dataframe("split(100)/train")

In [20]:
df_test=prepare_dataframe("split(100)/test")

In [21]:
df_val=prepare_dataframe("split(100)/val")

In [22]:
def datagenerator(input_dataframe, img_size, batch_size):
    datagen = ImageDataGenerator(rescale=1./255)
    
    img = datagen.flow_from_dataframe(
        dataframe=input_dataframe,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='input',
        shuffle=True,
        workers=8,
        use_multiprocessing=True 
    )
    
    return img

In [23]:
img_size = 64
batch_size = 16

In [24]:
train_img = datagenerator(df_train, img_size, batch_size)
val_img = datagenerator(df_val, img_size, batch_size)

Found 70 validated image filenames.
Found 15 validated image filenames.


In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_img = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode="input",
        shuffle=False,
        workers=8,
        use_multiprocessing=True 
    )

Found 15 validated image filenames.


In [26]:
def ms_ssim_loss(y_true, y_pred):
    ms_ssim = tf.image.ssim_multiscale(y_true, y_pred, max_val=1.0)
    loss = 1 - ms_ssim
    return loss

In [27]:
def ssim_loss(y_true, y_pred):
    ssim = tf.image.ssim(y_true, y_pred, max_val=1.0)
    loss = 1 - ssim
    return loss

In [41]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    
    def build_skip_autoencoder(input_img):
         x = input_img 
         x1 = x
         x = Conv2D(12, (3, 3), activation='relu', padding='same')(x)
         x = MaxPooling2D((2, 2), padding='same')(x)
         x2 = Conv2D(12, (1, 1), strides=2, activation='relu', padding='same')(x1)
         x = Concatenate()([x, x2])
         x = Conv2D(1, (3,3), activation='relu',padding='same')(x)
         encoded = MaxPooling2D((2,2), padding='same')(x)
        
         x = Conv2D(1,(3,3), activation='relu',padding='same')(encoded) 
         x = UpSampling2D((2,2))(x)
         x3 = x
         x = Conv2D(12, (3, 3), activation='relu', padding='same')(x)
         x = UpSampling2D((2, 2))(x)
         x4 = Conv2DTranspose(12, (1, 1), strides=2, activation='relu', padding='same')(x3)
         x = Concatenate()([x, x4])
         dec_output = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

         autoencoder = Model(input_img, dec_output)
         
         autoencoder.compile(optimizer='rmsprop', loss=lambda y_true, y_pred: ssim_loss(y_true, y_pred))
         autoencoder.summary()
   
         return autoencoder

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [42]:
input_img = Input(shape=(64, 64, 1))
autoencoder = build_skip_autoencoder(input_img)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 64, 64,    │        120 │ input_layer_4[0]… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 32, 32,    │          0 │ conv2d_24[0][0]   │
│ (MaxPooling2D)      │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 32, 32,    │         24 │ input_layer_4[0]… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 32, 32,    │          0 │ max_pooling2d_8[… │
│ (Concatenate)       │ 24)               │            │ conv2d_25[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_26 (Conv2D)  │ (None, 32, 32, 1) │        217 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 16, 16, 1) │          0 │ conv2d_26[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 16, 16, 1) │         10 │ max_pooling2d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_8     │ (None, 32, 32, 1) │          0 │ conv2d_27[0][0]   │
│ (UpSampling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_28 (Conv2D)  │ (None, 32, 32,    │        120 │ up_sampling2d_8[… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_9     │ (None, 64, 64,    │          0 │ conv2d_28[0][0]   │
│ (UpSampling2D)      │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_4  │ (None, 64, 64,    │         24 │ up_sampling2d_8[… │
│ (Conv2DTranspose)   │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 64, 64,    │          0 │ up_sampling2d_9[… │
│ (Concatenate)       │ 24)               │            │ conv2d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_29 (Conv2D)  │ (None, 64, 64, 1) │        217 │ concatenate_5[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 732 (2.86 KB)

 Trainable params: 732 (2.86 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
early_stopping = EarlyStopping(
             monitor='val_loss',
             patience=10,
            verbose=1,
            restore_best_weights=True       
)

In [37]:
import datetime
current_time = datetime.datetime.now()
print(current_time)

2024-08-21 16:14:05.639261


In [ ]:
history = autoencoder.fit(train_img, 
                          epochs=10, 
                          batch_size=16,
                          validation_data=val_img
                          )

Epoch 1/10


In [ ]:
end_time = datetime.datetime.now()
print(end_time)

In [ ]:
#autoencoder.save("5k_2L_ms-ssim_loss_12f.keras")